## `Flight Delay Prediction`
### **Part 2:** Data Preprocessing

-- This is just a mass cleanup of the data. The data will be polished and cleaned further in Part 3: EDA.

Ali Bahrami

In [56]:
import numpy as np
import pandas as pd

In [57]:
# Read from extracted data
df_flights_raw = pd.read_csv("data/raw/flights_raw.csv")
df_passengers_raw = pd.read_csv("data/raw/passengers_raw.csv")
df_fuel_raw = pd.read_csv("data/raw/fuel_raw.csv")

In [78]:
# Take copy to measure data loss after clean-up
df_flights = df_flights_raw.copy()
df_passengers = df_passengers_raw.copy()
df_fuel = df_fuel_raw.copy()

### Missing Values Treatment

In [79]:
df_flights.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-05-04,DL,DL_CODESHARE,DL,5535,EV,N754EV,5535,10721,BOS,...,474.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-12-09,UA,UA,UA,1754,UA,N828UA,1754,13495,MSY,...,1167.0,0.0,0.0,178.0,0.0,0.0,NaN,NaN,NaN,NaN
2,2019-12-20,AA,AA,AA,1988,AA,N763US,1988,11057,CLT,...,1095.0,19.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,2019-07-14,DL,DL,DL,2230,DL,N961DL,2230,10397,ATL,...,1027.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-04-08,AA,AA,AA,1904,AA,N679AW,1904,14107,PHX,...,735.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Drop cancelled and diverted flights, given that the focus of the project is only on flights delays


In [80]:
df_flights = df_flights[(df_flights['cancelled'] != 1)&(df_flights['diverted']!=1)]

Drop unnecessary columns:
- No need for mkt_carrier data as there is operating carrier
- cancellation codes are not needed.
- flight numbers are not needed.
- airport ids are not needed.
- other columns with more than 90% of data missing 

In [81]:
cols = ['cancellation_code' , 'mkt_carrier','op_unique_carrier', 'branded_code_share', 'mkt_carrier_fl_num', 'op_carrier_fl_num', 'cancelled','diverted','first_dep_time', 'origin_airport_id', 'dest_airport_id', 'flights','total_add_gtime', 'longest_add_gtime', 'no_name']
df_flights = df_flights.drop(columns=cols)

In [82]:
df_flights.isnull().sum()

fl_date                     0
mkt_unique_carrier          0
tail_num                    0
origin                      0
origin_city_name            0
dest                        0
dest_city_name              0
crs_dep_time                0
dep_time                    0
dep_delay                  55
taxi_out                   78
wheels_off                 78
wheels_on                  78
taxi_in                    78
crs_arr_time                0
arr_time                    0
arr_delay                  27
dup                         0
crs_elapsed_time            0
actual_elapsed_time         0
air_time                   78
distance                    0
carrier_delay          158311
weather_delay          158311
nas_delay              158311
security_delay         158311
late_aircraft_delay    158311
dtype: int64

The columns with reasons of delay (carrier delay, weather delay, etc.) all seem to similar amounts of missing data: 158311

which is more than 50% of the entire dataset. 

I will fill them with zero for now, but will drop them during the modeling phase.

In [83]:
# Fill na values in appropriate columns with zero.
# i.e. security_delay either has values in minutes or na values which would be zero

columns_fill = ['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']
df_flights.update(df_flights[columns_fill].fillna(0))

In [84]:
# Drop any sporadic nan values (i.e. ghost flights with no air time) (207 values)
df_flights.dropna(axis=0, inplace=True)

In [85]:
df_flights.isnull().sum()

fl_date                0
mkt_unique_carrier     0
tail_num               0
origin                 0
origin_city_name       0
dest                   0
dest_city_name         0
crs_dep_time           0
dep_time               0
dep_delay              0
taxi_out               0
wheels_off             0
wheels_on              0
taxi_in                0
crs_arr_time           0
arr_time               0
arr_delay              0
dup                    0
crs_elapsed_time       0
actual_elapsed_time    0
air_time               0
distance               0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
dtype: int64

## Preliminary Data Manipulation and Feature Engineering

In [86]:
# The airlines names are given as abbreviations only, import the full name from the files.
airlines = pd.read_csv("data/airlines.csv")
abbr_airlines = airlines.set_index('IATA_CODE')['AIRLINE'].to_dict()

# Add the full airline names to the dataframe
df_flights['mkt_carrier_name'] = df_flights['mkt_unique_carrier'].replace(abbr_airlines)

In [87]:
df_flights['mkt_carrier_name'].unique()

array(['Delta Air Lines', 'United Airlines', 'American Airlines',
       'Southwest Airlines', 'Alaska Airlines', 'Frontier Airlines',
       'Hawaiian Airlines', 'Spirit Airlines', 'JetBlue Airways',
       ' Allegiant Air', 'Virgin America'], dtype=object)

Day of Week GLOSSARY

0 = Monday

1 = Tuesday

2 = Wednesday

3 = Thursday

4 = Friday

5 = Saturday

6 = Sunday

In [89]:
# Add separate columns for year, month, day of month, and day of week
df_flights['fl_date'] = pd.to_datetime(df_flights['fl_date'],format='%Y-%m-%d')
df_flights['year'] = pd.DatetimeIndex(df_flights['fl_date']).year
df_flights['month'] = pd.DatetimeIndex(df_flights['fl_date']).month
df_flights['day_of_month'] = pd.DatetimeIndex(df_flights['fl_date']).day
df_flights['day_of_week'] = pd.DatetimeIndex(df_flights['fl_date']).dayofweek

Divide the flight into short, medium, and long haul flights based on air time

In [90]:

def haul(df, col):
    '''Add a column to determine flight haul (short, medium, long) based on air_time'''
    length=[]
    for i in df[col]:
        if i < (3*60): # up to 3 hours
            length.append('short')
        elif (i >= (3*60)) and (i < (6*60)): # 3-6 hours
            length.append('medium')
        elif i >= (6*60):
            length.append('long') # 6+ hours
    df_flights['haul_length'] = length

haul(df_flights, 'air_time')

In [91]:
# Add flight route
df_flights['route'] = df_flights['origin'] + "-" + df_flights['dest']

### Binary Classification
Classify the flights in a new column 'flight_delayed' into:

delayed = 1

not delayed = 0

In [94]:
status = []

for value in df_flights['arr_delay']:
    if value <= 0:
        status.append(0)
    else:
        status.append(1)
df_flights['flight_delayed'] = status


In [95]:
df_flights.head() 

,fl_date,mkt_unique_carrier,tail_num,origin,origin_city_name,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,...,security_delay,late_aircraft_delay,mkt_carrier_name,year,month,day_of_month,day_of_week,haul_length,route,flight_delayed
0,2018-05-04,DL,N754EV,BOS,"Boston, MA",RIC,"Richmond, VA",1455,1453.0,-2.0,...,0.0,0.0,Delta Air Lines,2018,5,4,4,short,BOS-RIC,0
1,2019-12-09,UA,N828UA,MSY,"New Orleans, LA",EWR,"Newark, NJ",1515,1757.0,162.0,...,0.0,0.0,United Airlines,2019,12,9,0,short,MSY-EWR,1
2,2019-12-20,AA,N763US,CLT,"Charlotte, NC",SAT,"San Antonio, TX",1610,1630.0,20.0,...,0.0,0.0,American Airlines,2019,12,20,4,short,CLT-SAT,1
3,2019-07-14,DL,N961DL,ATL,"Atlanta, GA",PWM,"Portland, ME",2114,2114.0,0.0,...,0.0,0.0,Delta Air Lines,2019,7,14,6,short,ATL-PWM,0
4,2019-04-08,AA,N679AW,PHX,"Phoenix, AZ",BOI,"Boise, ID",2021,2015.0,-6.0,...,0.0,0.0,American Airlines,2019,4,8,0,short,PHX-BOI,0


## Passengers Dataset
No missing values in Passengers

In [96]:
df_passengers.isnull().sum()

departures_scheduled     0
departures_performed     0
payload                  0
seats                    0
passengers               0
freight                  0
mail                     0
distance                 0
ramp_to_ramp             0
air_time                 0
unique_carrier           0
airline_id               0
unique_carrier_name      0
region                   0
carrier                  0
carrier_name             0
carrier_group            0
carrier_group_new        0
origin_airport_id        0
origin_city_market_id    0
origin                   0
origin_city_name         0
origin_country           0
origin_country_name      0
dest_airport_id          0
dest_city_market_id      0
dest                     0
dest_city_name           0
dest_country             0
dest_country_name        0
aircraft_group           0
aircraft_type            0
aircraft_config          0
year                     0
month                    0
distance_group           0
class                    0
d

## Fuel Consumption Dataset
Drop rows with missing data in fuel consumption (25 values in 3035 rows)

In [97]:
df_fuel = df_fuel.dropna()

## Data Loss

In [98]:
# Data Loss

# Flights
flights_raw_len = len(df_flights_raw)
flights_clean_len = len(df_flights)
flights_data_loss = round(((flights_raw_len - flights_clean_len)/flights_raw_len  * 100), 2)
print(f'Flights data lost in percent: {flights_data_loss}%')

# Passengers
passengers_raw_len = len(df_passengers_raw)
passengers_clean_len = len(df_passengers)
passengers_data_loss = round(((passengers_raw_len - passengers_clean_len)/passengers_raw_len  * 100), 2)
print(f'Passengers data lost in percent: {passengers_data_loss}%')

# Fuel Consumption
fuel_raw_len = len(df_fuel_raw)
fuel_clean_len = len(df_fuel)
fuel_data_loss = round(((fuel_raw_len - fuel_clean_len)/fuel_raw_len  * 100), 2)
print(f'Fuel data lost in percent: {fuel_data_loss}%')

Flights data lost in percent: 2.06%
Passengers data lost in percent: 0.0%
Fuel data lost in percent: 0.33%


## Export Cleaned Data

In [99]:
df_flights.to_csv("data/flights.csv", index=False)
df_passengers.to_csv("data/passengers.csv", index=False)
df_fuel.to_csv("data/fuel.csv", index=False)